In [50]:
import pandas as pd 


In [5]:
data = pd.read_csv('titanic_data.csv')
data = data.reset_index()

In [6]:
data = data.drop(['Cabin','Name','Ticket'],1)

In [7]:
data.head()

,index,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,1,0,3,male,22.0,1,0,7.2500,S
1,1,2,1,1,female,38.0,1,0,71.2833,C
2,2,3,1,3,female,26.0,0,0,7.9250,S
3,3,4,1,1,female,35.0,1,0,53.1000,S
4,4,5,0,3,male,35.0,0,0,8.0500,S


In [13]:
data.isnull().sum()

index            0
PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [8]:
data.count()

index          891
PassengerId    891
Survived       891
Pclass         891
Sex            891
Age            714
SibSp          891
Parch          891
Fare           891
Embarked       889
dtype: int64

In [9]:
from sklearn import preprocessing

## Imputer for null handeling

In [14]:
imp = preprocessing.Imputer(strategy='mean')

In [15]:
imp.fit(data[['Age']])

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [18]:
imp_age = imp.transform(data[['Age']])  # here in imp_age nas are replaced with mean

In [19]:
data['Age'] = imp_age

In [20]:
data.count()

index          891
PassengerId    891
Survived       891
Pclass         891
Sex            891
Age            891
SibSp          891
Parch          891
Fare           891
Embarked       889
dtype: int64

## Label encoder

In [21]:
data[-3:]

,index,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
888,888,889,0,3,female,29.699118,1,2,23.45,S
889,889,890,1,1,male,26.000000,0,0,30.00,C
890,890,891,0,3,male,32.000000,0,0,7.75,Q


In [24]:
lab_encode = preprocessing.LabelBinarizer()

In [28]:
data['Sex'] = lab_encode.fit_transform(data['Sex']) #transform string to categorical variable 
                                                    # with int index

In [29]:
data[-3:]

,index,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
888,888,889,0,3,0,29.699118,1,2,23.45,S
889,889,890,1,1,1,26.000000,0,0,30.00,C
890,890,891,0,3,1,32.000000,0,0,7.75,Q


## Label one-hot

In [31]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [32]:
data['Embarked'] = data['Embarked'].fillna('S') # replaces NAS with a specified str value

In [33]:
lab_bins = preprocessing.LabelBinarizer()

In [34]:
encode_embarks = lab_bins.fit_transform(data['Embarked']) #creaes dummy variaables

In [35]:
encode_embarks.shape

(891, 3)

In [39]:
print(data['Embarked'][-3:])
print(encode_embarks[-3:,:])


888    S
889    C
890    Q
Name: Embarked, dtype: object
[[0 0 1]
 [1 0 0]
 [0 1 0]]


In [40]:
lab_bins.classes_

array(['C', 'Q', 'S'],
      dtype='<U1')

In [41]:
data_with_embark = data.join( pd.DataFrame(encode_embarks, columns=lab_bins.classes_) )

In [42]:
data_with_embark[-2:]

,index,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,C,Q,S
889,889,890,1,1,1,26.0,0,0,30.00,C,1,0,0
890,890,891,0,3,1,32.0,0,0,7.75,Q,0,1,0


## Structure X and y data

In [51]:
X = data_with_embark.drop(['PassengerId',
                           'Survived',
                          'Embarked',
                          ], axis = 1)

In [44]:
X[:10]

,index,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
0,0,3,1,22.000000,1,0,7.2500,0,0,1
1,1,1,0,38.000000,1,0,71.2833,1,0,0
2,2,3,0,26.000000,0,0,7.9250,0,0,1
3,3,1,0,35.000000,1,0,53.1000,0,0,1
4,4,3,1,35.000000,0,0,8.0500,0,0,1
5,5,3,1,29.699118,0,0,8.4583,0,1,0
6,6,1,1,54.000000,0,0,51.8625,0,0,1
7,7,3,1,2.000000,3,1,21.0750,0,0,1
8,8,3,0,27.000000,0,2,11.1333,0,0,1
9,9,2,0,14.000000,1,0,30.0708,1,0,0


In [46]:
Y = data_with_embark['Survived']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [52]:
from sklearn.model_selection import train_test_split  #very frequently used

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)

In [54]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

In [55]:
rf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [56]:
rf.score(X_test,Y_test)  #check scor method of RandomForest

0.8067796610169492

# Grid Search

In [57]:
from sklearn.model_selection import GridSearchCV

In [58]:

param_grid = { 
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']}

In [63]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, n_jobs=-1,verbose=2) #-1 use all cores of your cpu
CV_rfc.fit(X_train, Y_train)
print(CV_rfc.best_params_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[CV] max_features=auto, n_estimators=200 .............................
[CV] max_features=auto, n_estimators=200 .............................
[CV] max_features=auto, n_estimators=200 .............................
[CV] max_features=auto, n_estimators=200 .............................
[CV] .............. max_features=auto, n_estimators=200, total=   0.6s
[CV] max_features=auto, n_estimators=200 .............................
[CV] .............. max_features=auto, n_estimators=200, total=   0.6s
[CV] max_features=auto, n_estimators=200 .............................
[CV] .............. max_features=auto, n_estimators=200, total=   0.6s
[CV] max_features=auto, n_estimators=200 .............................
[CV] .............. max_features=auto, n_estimators=200, total=   0.6s
[CV] max_features=auto, n_estimators=200 .............................
[CV] .............. max_features=auto, n_estimators=200, total=   0.7s
[CV] ...........

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.8s


[CV] max_features=sqrt, n_estimators=700 .............................
[CV] .............. max_features=sqrt, n_estimators=700, total=   2.3s
[CV] max_features=sqrt, n_estimators=700 .............................
[CV] .............. max_features=sqrt, n_estimators=700, total=   2.3s
[CV] max_features=sqrt, n_estimators=700 .............................
[CV] .............. max_features=sqrt, n_estimators=700, total=   2.3s
[CV] max_features=log2, n_estimators=200 .............................
[CV] .............. max_features=sqrt, n_estimators=700, total=   2.3s
[CV] max_features=log2, n_estimators=200 .............................
[CV] .............. max_features=log2, n_estimators=200, total=   0.6s
[CV] max_features=log2, n_estimators=200 .............................
[CV] .............. max_features=log2, n_estimators=200, total=   0.6s
[CV] max_features=log2, n_estimators=200 .............................
[CV] .............. max_features=log2, n_estimators=200, total=   0.6s
[CV] m

[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   22.8s finished


{'max_features': 'log2', 'n_estimators': 700}


In [62]:
CV_rfc.best_score_

0.82379349046015715

In [65]:
rfbp = RandomForestClassifier(**CV_rfc.best_params_)

In [66]:
rfbp.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [67]:
 rfbp.score(X_test,Y_test)

0.81694915254237288

In [68]:
Y_test_pred = rfbp.predict(X_test)

In [70]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_test_pred)

array([[160,  15],
       [ 39,  81]])